In [ ]:
import numpy as np
import glob
import os
import matplotlib.pyplot as plt

In [ ]:
from read_write_model import read_images_binary

In [ ]:
global_path = "/home/riuclement/Documents/colmap_exp/sub80_4/"
output_gen_path = global_path + "OUT_{}_{}/"
path_out = global_path + "{}.npy"

In [ ]:
def get_folder_info(folder_path):
    infos = folder_path.split("OUT")[-1].split("_")
    noise_value = infos[1]
    outlier_value = infos[2].strip("/")
    return np.float64(noise_value), np.float64(outlier_value)

In [ ]:
def format_value_for_filename(numerical_value_):
    if numerical_value_ == 0:
        return "0.0"
    else:
        formated_value = "%.1f" % numerical_value_
        if formated_value.startswith("0."):
            return formated_value[1:]
        return formated_value

In [ ]:
def read_time_values(time_path):
    time = np.zeros(2)
    if os.path.exists(time_path):
        val = np.loadtxt(time_path)
        if val.shape[0] == 2:
            time = val
    return time

In [ ]:
std_values = [float(i/2) for i in range(13)]
outlier_values = [float(i/10) for i in range(10)]
num_images = 50
image_nums = [1 + i for i in range(num_images)]
num_trial = 10
algorithms = ["ransac", "acransac", "fastac", "lrt"]

In [ ]:
time_values = np.zeros((2, num_trial, len(algorithms), len(std_values), len(outlier_values)))
precision_values = np.zeros((num_trial, len(algorithms), len(std_values), len(outlier_values)))
recall_values = np.zeros((num_trial, len(algorithms),  len(std_values), len(outlier_values)))
images_seen_values = np.zeros((num_trial, len(algorithms), len(std_values), len(outlier_values)))

In [ ]:
for i_std, std_val in enumerate(std_values):
    std_val_filename = format_value_for_filename(std_val)
    for i_outlier, outlier_val in enumerate(outlier_values):
        outlier_val_filename = format_value_for_filename(outlier_val)
        folder_path = output_gen_path.format(std_val_filename, outlier_val_filename)
        for i_trial in range(num_trial):
            inlier_outlier_path = folder_path + "inlier_outlier_{}.txt".format(i_trial)
            if os.path.exists(inlier_outlier_path):
                inlier_outlier_data = np.loadtxt(inlier_outlier_path)
                for i_algo, algo_name in enumerate(algorithms):
                        time_path = folder_path + algo_name + "_{}".format(i_trial) + "_time.txt"
                        time = read_time_values(time_path)
                        time_values[:, i_trial, i_algo, i_std, i_outlier] = time
                        image_path = folder_path + algo_name + "_{}".format(i_trial) + "_images.bin"
                        if os.path.exists(image_path):
                            images_result = read_images_binary(image_path)
                            num_true_positive = 0
                            num_estimated_positive = 0
                            num_GT_inliers = 0
                            num_images_seen = 0
                            for i_image, image_val in enumerate(image_nums):
                                if (image_val in images_result.keys()):
                                    num_images_seen += 1
                                    num_true_positive += np.sum(inlier_outlier_data[np.where(inlier_outlier_data[:, 0] == image_val)][images_result[image_val].point3D_ids > 0, 2])
                                    num_estimated_positive += np.sum(images_result[image_val].point3D_ids > 0)
                                num_GT_inliers += np.sum(inlier_outlier_data[np.where(inlier_outlier_data[:, 0] == image_val)][:, 2])
                            if num_estimated_positive > 0:
                                precision = num_true_positive / num_estimated_positive
                            if num_GT_inliers > 0:
                                recall = num_true_positive / num_GT_inliers

                            precision_values[i_trial, i_algo, i_std, i_outlier] = precision
                            recall_values[i_trial, i_algo, i_std, i_outlier] = recall
                            images_seen_values[i_trial, i_algo, i_std, i_outlier] = num_images_seen

In [ ]:
image_path

In [ ]:
precision_mean = precision_values.mean(axis=0)
recall_mean = recall_values.mean(axis=0)
time_mean = time_values.mean(axis=1)
images_seen_mean = images_seen_values.mean(axis=0)

In [ ]:
display_style = {
    "alpha" : 1.0,
    "colors" : {
        "ransac" : 'b',
        "acransac" : 'r',
        "fastac" : 'y',
        "lrt" : 'g',
    },
    "linestyles" : {
        "ransac" : '-',
        "acransac" : '-',
        "fastac" : '-',
        "lrt" : '-',
    },
}

In [ ]:
def print_image(algorithms_, outlier_values_, std_values_,
                display_style_,
                values_to_print_, name_value_print_,
                save_=False):
    legend = "{}."
    xlabel_n = "Noise std value (in pixel)"
    xlabel_o = "Outlier ratio value"
    generic_title = "{} for outlier ratio of {}."
    generic_suptitle = "{} for algorithms {}.".format("{}", " ".join(algorithms_))


    fig, axs = plt.subplots(1, len(outlier_values_), figsize=(10 * len(outlier_values_), 10), sharey=True)
    fig.suptitle(generic_suptitle.format(name_value_print_))
    lines = []
    legends = []

    for i_outlier, outlier_val in enumerate(outlier_values_):
        for i_algo, algo_name in enumerate(algorithms_):
            line = axs[i_outlier].plot(std_values_, values_to_print_[i_algo, :, i_outlier],
                                       color=display_style_["colors"][algo_name],
                                       linestyle=display_style_["linestyles"][algo_name],
                                       alpha=display_style_["alpha"])
            lines.append(line)
            legends.append(legend.format(algo_name))

    axs[i_outlier].legend(lines, legends)
    if save_:
        plt.savefig(save_, bbox_inches='tight')
    else:
        plt.show()

In [ ]:
print_image(algorithms, outlier_values, std_values, display_style, precision_mean, "Precision")
print_image(algorithms, outlier_values, std_values, display_style, recall_mean, "Recall")
print_image(algorithms, outlier_values, std_values, display_style, images_seen_mean, "Number of images seen")

In [ ]:
def print_time_image(algorithms_, outlier_values_, std_values_,
                display_style_,
                values_to_print_, name_value_print_,
                save_=False):
    legend = "{}."
    xlabel_n = "Noise std value (in pixel)"
    xlabel_o = "Outlier ratio value"
    generic_title = "{} for outlier ratio of {}."
    generic_suptitle = "{} for algorithms {}.".format("{}", " ".join(algorithms_))


    fig, axs = plt.subplots(1, len(outlier_values_), figsize=(10 * len(outlier_values_), 10), sharey=True)
    fig.suptitle(generic_suptitle.format(name_value_print_))
    lines = []
    legends = []

    for i_outlier, outlier_val in enumerate(outlier_values_):
        for i_algo, algo_name in enumerate(algorithms_):
            line = axs[i_outlier].plot(std_values_, values_to_print_[0, i_algo, :, i_outlier],
                                       color=display_style_["colors"][algo_name],
                                       linestyle="-",
                                       alpha=display_style_["alpha"])
            lines.append(line)
            legends.append("Global time for " + legend.format(algo_name))
            line = axs[i_outlier].plot(std_values_, values_to_print_[1, i_algo, :, i_outlier],
                                       color=display_style_["colors"][algo_name],
                                       linestyle="--",
                                       alpha=display_style_["alpha"])
            lines.append(line)
            legends.append("Ransac time for " + legend.format(algo_name))

    axs[i_outlier].legend(lines, legends)
    if save_:
        plt.savefig(save_, bbox_inches='tight')
    else:
        plt.show()

In [ ]:
print_time_image(algorithms, outlier_values, std_values, display_style, time_mean, "Time")

In [ ]:
np.save(path_out.format("precision_array"), precision_values)
np.save(path_out.format("recall_array"), recall_values)
np.save(path_out.format("image_seen_array"), images_seen_values)
np.save(path_out.format("time_array"), time_values)